In [1]:
import os, gc, torch, PIL, pickle,json,io

import pandas as pd
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from utils.plot import plot_losses, plot_train_val_evaluators
from PIL import Image
from models.load import get_model_label, get_model_name, get_current_epoch, get_dataset_label, get_trained_model

from data.constants import DEFAULT_REFLACX_LABEL_COLS

import utils.print as print_f
from utils.init import reproducibility, clean_memory_get_device

# from models.load import TrainedModels
## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    # mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    # mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    # resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    # resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    # densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    # densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    # efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    # efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    # efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    # efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    convnext_base_with_fix_silent = "val_lesion-detection_ap_0_2405_test_lesion-detection_ap_0_2543_epoch19_03-24-2023 13-57-29_lesion_dsetection_with_fixation_convnext_base_silent_report"
    convnext_base_with_fix_full = "val_lesion-detection_ap_0_2602_test_lesion-detection_ap_0_2499_epoch22_03-24-2023 04-42-21_lesion_dsetection_with_fixation_convnext_base_full_report"
    # vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    # vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    # regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    # regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [4]:
naming_map = {
    # TrainedModels.mobilenet_baseline: "mobilenet_baseline",
    # TrainedModels.mobilenet_with_fix: "mobilenet_with_fix",
    # TrainedModels.resnet18_baseline: "resnet18_baseline",
    # TrainedModels.resnet18_with_fix: "resnet18_with_fix",
    # TrainedModels.densenet161_baseline: "densenet161_baseline",
    # TrainedModels.densenet161_with_fix: "densenet161_with_fix",
    # TrainedModels.efficientnet_b5_baseline: "efficientnet_b5_baseline",
    # TrainedModels.efficientnet_b5_with_fix: "efficientnet_b5_with_fix",
    # TrainedModels.efficientnet_b0_baseline: "efficientnet_b0_baseline",
    # TrainedModels.efficientnet_b0_with_fix: "efficientnet_b0_with_fix",
    TrainedModels.convnext_base_with_fix: "convnext_base_with_fix",
    TrainedModels.convnext_base_baseline: "convnext_base_baseline",
    TrainedModels.convnext_base_with_fix_silent: "convnext_base_with_fix_silent",
    TrainedModels.convnext_base_with_fix_full:"convnext_base_with_fix_full",
    # TrainedModels.vgg16_with_fix: "vgg16_with_fix",
    # TrainedModels.vgg16_baseline: "vgg16_baseline",
    # TrainedModels.regnet_y_8gf_with_fix: "regnet_y_8gf_with_fix",
    # TrainedModels.regnet_y_8gf_baseline: "regnet_y_8gf_baseline",
}


In [5]:
score_thrs="0.05"

In [6]:
eval_ss = []

for select_model in TrainedModels:
    model_path = select_model.value
    for l in DEFAULT_REFLACX_LABEL_COLS + ["all"]:
        eval_df = pd.read_csv(
            os.path.join(
                "./eval_results",
                f"{model_path}_{l}_{score_thrs}.csv",
            ),
            index_col=0,
        )
        eval_s = eval_df.iloc[0]
        eval_s["model"] = naming_map[select_model]
        eval_s["lesion"] = l
        eval_s["score_threshold"] = score_thrs
        states = json.loads(eval_s["coco_states"])
        for s in states:
            eval_s[s["p_title_str"]] = s["mean_s"]
        eval_ss.append(eval_s)


In [7]:
eval_df = pd.DataFrame(eval_ss)

In [8]:
eval_df.head(5)

,num_fps,num_fns,num_tps,coco_states,Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",model,...,Average Precision (AP) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= small | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 10 ]
0,1479,37,162,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.497487,0.683082,0.820965,0.878734,0.720067,convnext_base_with_fix,...,-1,-1,0.132946,0.133166,0.464322,0.616080,0.663819,-1,-1,0.616080
0,1035,70,207,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.552604,0.756576,0.868258,0.975218,0.788164,convnext_base_with_fix,...,-1,-1,0.279736,0.282310,0.561372,0.610469,0.613718,-1,-1,0.610469
0,731,123,214,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.561820,0.724401,0.880658,0.968533,0.783853,convnext_base_with_fix,...,-1,-1,0.154392,0.209199,0.424629,0.457270,0.462018,-1,-1,0.457270
0,1141,96,221,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.426994,0.596501,0.771134,0.900766,0.673849,convnext_base_with_fix,...,-1,-1,0.118583,0.185174,0.476025,0.507886,0.511672,-1,-1,0.507886
0,400,259,145,"[{""ap"": 1, ""iouThr"": null, ""areaRng"": ""all"", ""...",0.448736,0.654635,0.850187,0.972856,0.731604,convnext_base_with_fix,...,-1,-1,0.068103,0.129208,0.208663,0.215347,0.219307,-1,-1,0.215347


In [10]:
eval_df = eval_df[[
    "model",
    "lesion",
    "num_tps",
    "num_fps",
    "num_fns",
    "Sensitivity@ [avgFP=0.5]",
    "Sensitivity@ [avgFP=1]",
    "Sensitivity@ [avgFP=2]",
    "Sensitivity@ [avgFP=4]",
    "mFROC@[0.5,1,2,4]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 10 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
]]


In [11]:
eval_df = eval_df.rename(columns={"num_tps": "#TP @[ IoU=0.50      | area=   all | maxDets= 30 ]", "num_fps": "#FP @[ IoU=0.50      | area=   all | maxDets= 30 ]", "num_fns":"#FN @[ IoU=0.50      | area=   all | maxDets= 30 ]"})

In [12]:
eval_df

,model,lesion,#TP @[ IoU=0.50 | area= all | maxDets= 30 ],#FP @[ IoU=0.50 | area= all | maxDets= 30 ],#FN @[ IoU=0.50 | area= all | maxDets= 30 ],Sensitivity@ [avgFP=0.5],Sensitivity@ [avgFP=1],Sensitivity@ [avgFP=2],Sensitivity@ [avgFP=4],"mFROC@[0.5,1,2,4]",Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ]
0,convnext_base_with_fix,Pulmonary edema,162,1479,37,0.497487,0.683082,0.820965,0.878734,0.720067,0.663819,0.137965,0.033492,0.112745,0.132946,0.230238,0.118131,0.133166,0.464322,0.616080
0,convnext_base_with_fix,Enlarged cardiac silhouette,207,1035,70,0.552604,0.756576,0.868258,0.975218,0.788164,0.613718,0.279914,0.176505,0.270803,0.279736,0.394860,0.295134,0.282310,0.561372,0.610469
0,convnext_base_with_fix,Consolidation,214,731,123,0.561820,0.724401,0.880658,0.968533,0.783853,0.462018,0.154870,0.098484,0.149496,0.154392,0.287675,0.126044,0.209199,0.424629,0.457270
0,convnext_base_with_fix,Atelectasis,221,1141,96,0.426994,0.596501,0.771134,0.900766,0.673849,0.511672,0.118849,0.057768,0.115419,0.118583,0.217124,0.108835,0.185174,0.476025,0.507886
0,convnext_base_with_fix,Pleural abnormality,145,400,259,0.448736,0.654635,0.850187,0.972856,0.731604,0.219307,0.068358,0.066985,0.069036,0.068103,0.144135,0.054968,0.129208,0.208663,0.215347
0,convnext_base_with_fix,all,945,4758,589,0.429618,0.576799,0.739969,0.885052,0.657860,0.491254,0.150343,0.086347,0.142879,0.149067,0.256095,0.138168,0.186925,0.427443,0.477477
0,convnext_base_baseline,Pulmonary edema,148,1417,51,0.393635,0.627854,0.737780,0.868322,0.656898,0.575377,0.108671,0.027141,0.094279,0.107852,0.193290,0.099749,0.126131,0.438693,0.561809
0,convnext_base_baseline,Enlarged cardiac silhouette,197,968,80,0.582920,0.719384,0.887987,0.957086,0.786844,0.562455,0.265361,0.177151,0.255146,0.265404,0.392039,0.299780,0.264982,0.508303,0.562094
0,convnext_base_baseline,Consolidation,262,2002,75,0.485658,0.698177,0.872404,0.950527,0.751691,0.625223,0.165843,0.106757,0.152353,0.163420,0.269968,0.143708,0.206231,0.480712,0.588724
0,convnext_base_baseline,Atelectasis,139,460,178,0.333596,0.538954,0.748299,0.911969,0.633205,0.295584,0.074251,0.047411,0.074213,0.074251,0.151471,0.056871,0.154890,0.294637,0.295584


In [13]:
eval_df.to_csv("evaluation.csv")